<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [1]:
!nvidia-smi

Fri Feb 24 18:38:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-24 18:38:24--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-24 18:38:25 (73.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-24 18:38:28--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   755KB/s    in 0.8s    

2023-02-24 18:38:29 (755 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [6]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [7]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [9]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
test_data  = pd.read_csv("/content/test.csv")

In [11]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [12]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/2AO97o2a9D 1
#Metro still doesn't know when Blue &amp; Orange lines will reopen. Empty passenger car derailed ~ 5 a.m. NOT a new 7000-series. @CQnow #WMATA 1
Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/bgM4cSrbVd 1
CONFIRMED: Sanchez Hazard and Bolasie will be out for the rest of the season. https://t.co/7Ct01nEptL 1
Thanks a lot roadworks men cos a tube strike wasn't disruptive enough so having to walk the desolate route from Tottenham to ..... 1


## Split Data into Training and Validation Sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [18]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [19]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [20]:
text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1487,    8,  168,   24, 1405,    5, 6927,  124, 7486, 1115,  574,
        4630,   27, 1365,  308]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

British bake off was great pretty hilarious moments #mudslide
tf.Tensor(
[[ 0.02246429 -0.04697264  0.01227294 ... -0.03121109  0.04333926
  -0.00019959]
 [ 0.04533006 -0.00397581 -0.0185702  ...  0.00931972  0.02262839
  -0.0197663 ]
 [-0.01376421  0.00629078  0.04598371 ... -0.01485877  0.0155566
   0.01890535]
 ...
 [ 0.01424701 -0.02666005 -0.00709919 ...  0.00683524  0.04858894
   0.0216768 ]
 [ 0.01424701 -0.02666005 -0.00709919 ...  0.00683524  0.04858894
   0.0216768 ]
 [ 0.01424701 -0.02666005 -0.00709919 ...  0.00683524  0.04858894
   0.0216768 ]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
base_line_score = model_0.score(val_sentences,val_labels)

In [28]:
base_line_score

0.7926509186351706

In [29]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [30]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [31]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [32]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [33]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [36]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230224-183834
Epoch 1/5
215/215 [==============================] - 36s 152ms/step - loss: 0.6377 - accuracy: 0.6536 - val_loss: 0.5742 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4724 - accuracy: 0.8240 - val_loss: 0.4764 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3431 - accuracy: 0.8775 - val_loss: 0.4562 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.2587 - accuracy: 0.9085 - val_loss: 0.4600 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1984 - accuracy: 0.9330 - val_loss: 0.4696 - val_accuracy: 0.7848


In [37]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4696 - accuracy: 0.7848


[0.4695885479450226, 0.7847769260406494]

In [38]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [39]:
model_1_pred_probs[10]

array([0.21758638], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [41]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7880247798695493,
 'recall': 0.7847769028871391,
 'f1': 0.7823139263800303}

## Visualizing Learned Embeddings

In [43]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [46]:
embed_weights.shape

(10000, 128)

In [47]:
embed_weights

array([[-0.06864508, -0.02579901, -0.03377311, ..., -0.02094769,
         0.17260909, -0.0248047 ],
       [-0.07258762, -0.02783629, -0.05852331, ..., -0.03422688,
        -0.04343431, -0.02693323],
       [-0.04522338, -0.03535008, -0.04859307, ..., -0.04885658,
        -0.02673205, -0.0393073 ],
       ...,
       [ 0.02307084, -0.03365733,  0.04772821, ..., -0.03407582,
         0.00173529, -0.00776883],
       [ 0.04248806,  0.00704304, -0.02734864, ...,  0.02432572,
         0.07972275,  0.03453425],
       [-0.06070052,  0.04250436, -0.03178068, ..., -0.02614878,
        -0.02609077,  0.02151337]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [49]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [50]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230224-183924
Epoch 1/5
215/215 [==============================] - 15s 44ms/step - loss: 0.3778 - accuracy: 0.8324 - val_loss: 0.4622 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2470 - accuracy: 0.9009 - val_loss: 0.4680 - val_accuracy: 0.7927
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1800 - accuracy: 0.9311 - val_loss: 0.5960 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1324 - accuracy: 0.9510 - val_loss: 0.7398 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1012 - accuracy: 0.9599 - val_loss: 1.0578 - val_accuracy: 0.7612


In [51]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[0.01341213],
       [0.97285277],
       [0.99987113],
       [0.01518233],
       [0.00331236],
       [0.9999021 ],
       [0.9924878 ],
       [0.9999765 ],
       [0.99996805],
       [0.7419266 ]], dtype=float32)

In [52]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [53]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [54]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 76.11548556430446,
 'precision': 0.7607410097018414,
 'recall': 0.7611548556430446,
 'f1': 0.760689014360106}

### Model-3 GRU Powered RNN

In [55]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [56]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [57]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [58]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230224-184009
Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.2066 - accuracy: 0.9115 - val_loss: 0.6942 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.1002 - accuracy: 0.9663 - val_loss: 0.6262 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0841 - accuracy: 0.9723 - val_loss: 0.7686 - val_accuracy: 0.7769
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0676 - accuracy: 0.9740 - val_loss: 0.9606 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0577 - accuracy: 0.9743 - val_loss: 1.0022 - val_accuracy: 0.7677


In [59]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.01678198],
       [0.8120505 ],
       [0.99975795],
       [0.06855877],
       [0.00140588],
       [0.9997271 ],
       [0.9667712 ],
       [0.9999    ],
       [0.999866  ],
       [0.9573984 ]], dtype=float32)

In [60]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [61]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 76.77165354330708,
 'precision': 0.7764212819224684,
 'recall': 0.7677165354330708,
 'f1': 0.7696307116337471}

### Model-4 Bidirectional RNN

In [63]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [64]:
model_4.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                             

In [65]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [66]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230224-184033
Epoch 1/5
215/215 [==============================] - 19s 49ms/step - loss: 0.1211 - accuracy: 0.9508 - val_loss: 0.9614 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.0627 - accuracy: 0.9742 - val_loss: 1.1343 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0573 - accuracy: 0.9752 - val_loss: 1.2688 - val_accuracy: 0.7507
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0477 - accuracy: 0.9774 - val_loss: 1.5104 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0424 - accuracy: 0.9803 - val_loss: 1.6175 - val_accuracy: 0.7507


In [67]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 1s 4ms/step


In [68]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 75.06561679790026,
 'precision': 0.7502044394332691,
 'recall': 0.7506561679790026,
 'f1': 0.7499080251314462}

### Model-5: Conv1D 

In [69]:
embedding_text = embedding(text_vectorizer(['this is a test sentence for embedding']))

In [70]:
embedding_text

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.08600795,  0.01631243, -0.03170937, ..., -0.02751814,
          0.05902931, -0.0316139 ],
        [-0.0311207 , -0.04341822,  0.06094972, ..., -0.04119885,
         -0.03134639, -0.06237734],
        [-0.04356621, -0.02873161, -0.03112365, ..., -0.06071092,
          0.03407941, -0.01306849],
        ...,
        [-0.03145347, -0.0335195 , -0.04517306, ..., -0.02211529,
          0.02165598, -0.03727269],
        [-0.03145347, -0.0335195 , -0.04517306, ..., -0.02211529,
          0.02165598, -0.03727269],
        [-0.03145347, -0.0335195 , -0.04517306, ..., -0.02211529,
          0.02165598, -0.03727269]]], dtype=float32)>

In [71]:
conv_1d = layers.Conv1D(filters = 32,
kernel_size = 5,
activation = "relu",
padding = "same"
)

In [72]:
conv_1d_output = conv_1d(embedding_text)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_text.shape, conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 32]), TensorShape([1, 32]))

In [73]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.06302737, 0.04825911, 0.08418755, 0.0143825 , 0.16377845,
        0.04483651, 0.11332434, 0.07044762, 0.16334993, 0.10167639,
        0.01638198, 0.06639749, 0.0343513 , 0.04789306, 0.10277443,
        0.09729604, 0.00470778, 0.14300746, 0.05143233, 0.09150095,
        0.08523276, 0.0345701 , 0.12690938, 0.0542984 , 0.09252821,
        0.14955297, 0.06332579, 0.0186002 , 0.12559804, 0.04272746,
        0.04967258, 0.11366573]], dtype=float32)>

In [74]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size = 5,activation = "relu",padding = "valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name = "model_5_conv1d")

In [75]:
model_5.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [76]:
mode_5_history = model_5.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_5_conv")],
)

Saving TensorBoard log files to: model_logs/model_5_conv/20230224-184125
Epoch 1/5
215/215 [==============================] - 17s 64ms/step - loss: 0.1449 - accuracy: 0.9599 - val_loss: 0.8597 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0745 - accuracy: 0.9733 - val_loss: 0.9665 - val_accuracy: 0.7507
Epoch 3/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0620 - accuracy: 0.9772 - val_loss: 1.0587 - val_accuracy: 0.7441
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0554 - accuracy: 0.9793 - val_loss: 1.1364 - val_accuracy: 0.7428
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0516 - accuracy: 0.9791 - val_loss: 1.1326 - val_accuracy: 0.7362


In [77]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

24/24 [==============================] - 0s 2ms/step


In [78]:
calculate_results(val_labels,model_5_preds)

{'accuracy': 73.62204724409449,
 'precision': 0.7360073279430674,
 'recall': 0.7362204724409449,
 'f1': 0.7347108319328972}

### Model 6: TFHub Pretrained Sentence Encoder

In [79]:
import tensorflow_hub as hub

In [80]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [81]:
samples = embed([sample_sentence,"When you call universal sentence , turns into numbers"])

In [82]:
print(samples)

tf.Tensor(
[[-0.00165963  0.00170079  0.04788961 ...  0.00854664  0.00299551
  -0.04015192]
 [ 0.0568927  -0.08954596 -0.0283566  ... -0.00659815  0.05909768
  -0.01464346]], shape=(2, 512), dtype=float32)


In [83]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
            input_shape = [],
            dtype = tf.string,
            trainable = False,
            name = "USE"
        )

In [84]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(1,activation = "sigmoid"),
],name = "model_6_USE")

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [85]:
model_6.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ["accuracy"])

In [86]:
mode_6_history = model_6.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_6_use")],
)

Saving TensorBoard log files to: model_logs/model_6_use/20230224-184243
Epoch 1/5
215/215 [==============================] - 5s 14ms/step - loss: 0.6529 - accuracy: 0.7173 - val_loss: 0.6178 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.5841 - accuracy: 0.7898 - val_loss: 0.5663 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.5405 - accuracy: 0.7933 - val_loss: 0.5341 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 2s 12ms/step - loss: 0.5111 - accuracy: 0.7971 - val_loss: 0.5122 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4905 - accuracy: 0.7997 - val_loss: 0.4971 - val_accuracy: 0.7900


In [87]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_labels = tf.squeeze(tf.round(model_6_pred_probs))

24/24 [==============================] - 1s 9ms/step


In [88]:
calculate_results(val_labels,model_6_pred_labels)

{'accuracy': 79.00262467191601,
 'precision': 0.791135944218791,
 'recall': 0.7900262467191601,
 'f1': 0.7885699892643387}

### Model-7 : 10% of the Data

In [89]:
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels = train_labels[:train_10_percent_split]
print(train_sentences_10_percent)

['@mogacola @zamtriossu i screamed after hitting tweet'
 'Imagine getting flattened by Kurt Zouma'
 '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'
 "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet"
 'Somehow find you and I collide http://t.co/Ee8RpOahPk'
 '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao'
 'destroy the free fandom honestly'
 'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE'
 '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.'
 'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'
 'Lava Dragon Breeder! I just bred a Lava Dragon in DragonVale! Visit my park to check it out! http://t.co/QGum9x

In [90]:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(100,activation = "relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid"),
])
model_7.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [91]:
model_7.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_7 (Dense)             (None, 100)               51300     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,849,225
Trainable params: 51,401
Non-trainable params: 256,797,824
_________________________________________________________________


In [92]:
mode_7_history = model_7.fit(
    train_sentences_10_percent,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_7_use")],
)

Saving TensorBoard log files to: model_logs/model_7_use/20230224-184302
Epoch 1/5
22/22 [==============================] - 4s 81ms/step - loss: 0.6665 - accuracy: 0.6482 - val_loss: 0.6452 - val_accuracy: 0.6995
Epoch 2/5
22/22 [==============================] - 1s 26ms/step - loss: 0.5887 - accuracy: 0.8000 - val_loss: 0.5824 - val_accuracy: 0.7546
Epoch 3/5
22/22 [==============================] - 1s 26ms/step - loss: 0.5071 - accuracy: 0.8146 - val_loss: 0.5235 - val_accuracy: 0.7848
Epoch 4/5
22/22 [==============================] - 1s 26ms/step - loss: 0.4441 - accuracy: 0.8234 - val_loss: 0.4965 - val_accuracy: 0.7822
Epoch 5/5
22/22 [==============================] - 0s 22ms/step - loss: 0.4019 - accuracy: 0.8409 - val_loss: 0.4855 - val_accuracy: 0.7795


In [93]:
model_7_pred_probs = model_7.predict(val_sentences)

24/24 [==============================] - 1s 9ms/step


In [94]:
model_7_pred = tf.squeeze(tf.round(model_7_pred_probs))

In [95]:
calculate_results(val_labels,model_7_pred)

{'accuracy': 77.95275590551181,
 'precision': 0.7799065773530309,
 'recall': 0.7795275590551181,
 'f1': 0.7783167829714759}

### Uploading model in Tensorboard

In [96]:
!tensorboard dev upload --logdir ./model_logs \
--name "NLP Modelling" \
--description "Comparing different models"\
--one_shot

2023-02-24 18:43:11.153789: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 18:43:11.153892: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-24 18:43:11.153911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs

This TensorBoard wil

### Saving and Loading Model

In [97]:
model_6.save("model_6.h5")

In [98]:
loaded_model_6 = tf.keras.models.load_model(
  'model_6.h5',custom_objects = {"KerasLayer":hub.KerasLayer}
)

In [99]:
loaded_model_6.evaluate(val_sentences,val_labels)

24/24 [==============================] - 2s 19ms/step - loss: 0.4971 - accuracy: 0.7900


[0.4970538020133972, 0.7900262475013733]

## Most Wrong Examples

In [100]:
!wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
!unzip 08_model_6_USE_feature_extractor.zip

--2023-02-24 18:43:52--  https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960779165 (916M) [application/zip]
Saving to: ‘08_model_6_USE_feature_extractor.zip’

08_model_6_USE_feat 100%[===================>] 916.27M  23.7MB/s    in 46s     

2023-02-24 18:44:40 (19.9 MB/s) - ‘08_model_6_USE_feature_extractor.zip’ saved [960779165/960779165]

Archive:  08_model_6_USE_feature_extractor.zip
   creating: 08_model_6_USE_feature_extractor/
   creating: 08_model_6_USE_feature_extractor/assets/
   creating: 08_model_6_USE_feature_extractor/variables/
  inflating: 08_model_6_USE_feature_extractor/variables/variables.data-00000-of-00001  
  inflating: 08_model_6_USE_feature_extractor/variables/variables.index

In [101]:
model_6_pretrained = tf.keras.models.load_model("08_model_6_USE_feature_extractor")
model_6_pretrained.evaluate(val_sentences,val_labels)

24/24 [==============================] - 1s 20ms/step - loss: 0.4272 - accuracy: 0.8163


[0.42723122239112854, 0.8162729740142822]

In [102]:
pretrained_preds = model_6_pretrained.predict(val_sentences)
predicted_labels = tf.squeeze(tf.round(pretrained_preds))

24/24 [==============================] - 1s 9ms/step


In [103]:
val_df = pd.DataFrame({
    "text":val_sentences,
    "target":val_labels,
    "pred":predicted_labels,
    "pred_prob":tf.squeeze(pretrained_preds),
})

In [104]:
val_df

,text,target,pred,pred_prob
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808
...,...,...,...,...
757,That's the ultimate road to destruction,0,0.0,0.137074
758,@SetZorah dad why dont you claim me that mean ...,0,0.0,0.095865
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
760,Crack in the path where I wiped out this morni...,0,1.0,0.612631


In [105]:
most_wrong = val_df[val_df['target'] != val_df['pred']].sort_values("pred_prob",ascending = False)

In [106]:
most_wrong

,text,target,pred,pred_prob
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.910196
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
628,@noah_anyname That's where the concentration c...,0,1.0,0.852300
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.835454
251,@AshGhebranious civil rights continued in the ...,0,1.0,0.827213
...,...,...,...,...
411,@SoonerMagic_ I mean I'm a fan but I don't nee...,1,0.0,0.043919
233,I get to smoke my shit in peace,1,0.0,0.042087
38,Why are you deluged with low self-image? Take ...,1,0.0,0.038998
244,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.038949


## Visualizing Model's Wrong Predictions

In [108]:
for row in most_wrong[:10].itertuples():
  _,text,target,pred,pred_prob = row
  print("Sentence : ",text)
  print("pred : ",pred)
  print("Actual :",target)
  print("Prediction Probability : ",pred_prob)
  print('-----')

Sentence :  ? High Skies - Burning Buildings ? http://t.co/uVq41i3Kx2 #nowplaying
pred :  1.0
Actual : 0
Prediction Probability :  0.9101957678794861
-----
Sentence :  FedEx will no longer transport bioterror pathogens in wake of anthrax lab mishaps http://t.co/lHpgxc4b8J
pred :  1.0
Actual : 0
Prediction Probability :  0.8769821524620056
-----
Sentence :  @noah_anyname That's where the concentration camps and mass murder come in. 
 
EVERY. FUCKING. TIME.
pred :  1.0
Actual : 0
Prediction Probability :  0.8523000478744507
-----
Sentence :  Ashes 2015: AustraliaÛªs collapse at Trent Bridge among worst in history: England bundled out Australia for 60 ... http://t.co/t5TrhjUAU0
pred :  1.0
Actual : 0
Prediction Probability :  0.8354544639587402
-----
Sentence :  @AshGhebranious civil rights continued in the 60s. And what about trans-generational trauma? if anything we should listen to the Americans.
pred :  1.0
Actual : 0
Prediction Probability :  0.8272132873535156
-----
Sentence :  @So

In [115]:
test_dataset = pd.read_csv('/content/test.csv')

In [116]:
test_dataset

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [120]:
test_sentences = test_dataset['text'].to_list()

In [125]:
test_samples = random.sample(test_sentences,10)
for test_sample in test_samples:
  pred_prob = tf.squeeze(model_6_pretrained.predict([test_sample]))
  pred_labels = tf.round(pred_prob)
  print("Pred : ",int(pred_labels))
  print("Prob : ",pred_prob)
  print("Text : ",test_sample)
  print("------")

1/1 [==============================] - 0s 45ms/step
Pred :  0
Prob :  tf.Tensor(0.034338593, shape=(), dtype=float32)
Text :  You da One 

#MTVSummerStar #VideoVeranoMTV  #MTVHottest Britney Spears Lana Del Rey
------
1/1 [==============================] - 0s 39ms/step
Pred :  0
Prob :  tf.Tensor(0.10814762, shape=(), dtype=float32)
Text :  8' MTech Assisted Open RESCUE Pocket Knife - NEW BLUE MT-A801BL zix http://t.co/51n2rZEBis http://t.co/P4lNbjDo0x
------
1/1 [==============================] - 0s 42ms/step
Pred :  1
Prob :  tf.Tensor(0.95107114, shape=(), dtype=float32)
Text :  RT @LivingSafely: NWS posts Severe #Thunderstorm Warnings for parts of #AR #NC #OK. Seek strong shelter if at risk: http://t.co/kImr0l24Fb
------
1/1 [==============================] - 0s 60ms/step
Pred :  0
Prob :  tf.Tensor(0.030411383, shape=(), dtype=float32)
Text :  @MikeParrActor devastated your no longer in emmerdale best character with so much more to give #superbactor your going to be missed
------


array([[0.5199701 ],
       [0.8752513 ],
       [0.827833  ],
       ...,
       [0.9158134 ],
       [0.51764417],
       [0.672602  ]], dtype=float32)